In [1]:
import sys, platform
import pvlib
import PySAM
import bifacial_radiance as br
import pandas as pd

In [2]:
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("Pvlib version: ", pvlib.__version__)
print("PySAM version: ", PySAM.__version__)
print("bifacial_radiance version: ", br.__version__)

Working on a  Windows 10
Python version  3.11.7 | packaged by Anaconda, Inc. | (main, Dec 15 2023, 18:05:47) [MSC v.1916 64 bit (AMD64)]
Pandas version  2.1.0
Pvlib version:  0.10.4.dev9+ge92ee94
PySAM version:  5.1.0
bifacial_radiance version:  0.4.2+340.g7909811.dirty


In [3]:
NREL_API_KEY = None  # <-- please set your NREL API key here
# note you must use "quotes" around your key as it is a string.

if NREL_API_KEY is None:
       NREL_API_KEY = 'DEMO_KEY'  # OK for this demo, but better to get your own key

In [4]:
import pvlib

metdata, metadata = pvlib.iotools.get_psm3(
    latitude=44.25, longitude=-117.33999633789062,
    api_key=NREL_API_KEY,
    email='silvana.ovaitt@nrel.gov',  # <-- any email works here fine
    names='tmy', map_variables=True)
metadata

{'Source': 'NSRDB',
 'Location ID': '200667',
 'City': '-',
 'State': '-',
 'Country': '-',
 'Time Zone': -7,
 'Local Time Zone': -7,
 'Dew Point Units': 'c',
 'DHI Units': 'w/m2',
 'DNI Units': 'w/m2',
 'GHI Units': 'w/m2',
 'Temperature Units': 'c',
 'Pressure Units': 'mbar',
 'Wind Direction Units': 'Degrees',
 'Wind Speed Units': 'm/s',
 'Surface Albedo Units': 'N/A',
 'Version': '3.2.0',
 'latitude': 44.25,
 'longitude': -117.34,
 'altitude': 979}

In [5]:
import os
from pathlib import Path

testfolder = 'TEMP'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in TEMP


In [6]:
radObj = br.RadianceObj('Sim3',path=testfolder, hpc=True)

path = TEMP
Making path: images
Making path: objects
Making path: results
Making path: skies
Making path: EPWs
Making path: materials


In [7]:
# Some of the names changed internally. While bifacial_radiance updates their expected names, we are renaming the values here
metadata['timezone'] = metadata['Time Zone']
metadata['county'] = '-'
metadata['elevation'] = metadata['altitude']
metadata['state'] = metadata['State']
metadata['country'] = metadata['Country']
metdata['Albedo'] = metdata['albedo']

Use NSRDBWeatherData to enter data the downloaded data in dataframe and dictionary forma for meteorological data and metadata respectively

In [8]:
#starttime can be 'MM_DD', or 'MM_DD_HH'
metData = radObj.readWeatherData(metadata, metdata, starttime='11_08_09', endtime='11_08_11',coerce_year=2021, label='center')

8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for center labeled data, at exact timestamp in input Weather File


In [9]:
alb = 0.2
radObj.setGround(alb) 


Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.


In [10]:
import numpy as np
import bifacialvf

In [11]:
y=2
solposAzi = metData.solpos['azimuth'].iloc[0]
solposZen = metData.solpos['zenith'].iloc[0]
timezonesave = metData.timezone

tilt = np.round(metData.latitude)
if tilt > 40.0:
    tilt = 40.0

DD = bifacialvf.vf.rowSpacing(beta = tilt, 
                              sazm=180, lat = metData.latitude, 
                              lng = metData.longitude, 
                              tz = metData.timezone, 
                              hour = 9, 
                              minute = 0.0)
if (DD <= 0) or (DD > 3.725):
    DD = 3.725
    print("Cannot find ideal pitch for location, setting D to 3.725")

normalized_pitch = DD + np.cos(np.round(metData.latitude) / 180.0 * np.pi)
pitch_temp = normalized_pitch*y
print("PITCH_temp ", pitch_temp)


Cannot find ideal pitch for location, setting D to 3.725
PITCH_temp  8.888679600677303


In [12]:
# -- establish tracking angles
hub_height = 1.5
pitch = 5
sazm = 180  # Tracker axis azimuth
modulename = 'PVmodule'
fixed_tilt_angle = None
gcr = 2 / pitch


trackerParams = {'limit_angle':50,
             'angledelta':5,
             'backtrack':True,
             'gcr':gcr,
             'cumulativesky':False,
             'azimuth': sazm,
             'fixed_tilt_angle': fixed_tilt_angle
             }

In [13]:

setup = 1

if setup == 1:
    hub_height = 1.5
    pitch = 5
    sazm = 180  # Tracker axis azimuth
    modulename = 'PVmodule'
    bedsWanted = 3
    fixed_tilt_angle = None
if setup == 2:
    hub_height = 2.4
    pitch = 5
    sazm = 180
    modulename = 'PVmodule'
    bedsWanted = 3
    fixed_tilt_angle = None
if setup == 3:
    hub_height = 2.4
    pitch = 5
    sazm = 180
    modulename = 'PVmodule_1mxgap'
    bedsWanted = 3
    fixed_tilt_angle = None
if setup == 4:
    hub_height = 1.5
    pitch = 8
    sazm = 180
    modulename = 'PVmodule'
    bedsWanted = 6
    fixed_tilt_angle = None
if setup == 5:
    hub_height = 1.5
    pitch = 11
    sazm = 180
    modulename = 'PVmodule'
    bedsWanted = 9
    fixed_tilt_angle = None
if setup == 6:
    hub_height = 1.5
    #tilt = None # fixed
    sazm = 180
    pitchfactor = 1
    modulename = 'PVmodule'
    pitch = pitch_temp * pitchfactor
    bedsWanted = 3
    fixed_tilt_angle = tilt
if setup == 7:
    hub_height = 2.4 
    sazm = 180
    pitchfactor = 1
    pitch = pitch_temp * pitchfactor
    modulename = 'PVmodule'
    bedsWanted = 3
    fixed_tilt_angle = tilt
if setup == 8:
    hub_height = 2.4 
    sazm = 180
    pitchfactor = 1
    pitch = pitch_temp * pitchfactor
    modulename = 'PVmodule_1mxgap'
    bedsWanted = 3
    fixed_tilt_angle = tilt
if setup == 9:
    hub_height = 1.5 
    sazm = 180
    pitchfactor = 2
    pitch = pitch_temp * pitchfactor
    modulename = 'PVmodule'
    bedsWanted = 6
    fixed_tilt_angle = tilt
if setup == 10:
    hub_height = 2 
    sazm = 90
    pitch = 8.6 
    modulename = 'PVmodule'
    bedsWanted = 7
    xp = 8
    fixed_tilt_angle = 90


In [14]:
gcr = 2/pitch

# -- establish tracking angles
trackerParams = {'limit_angle':50,
                 'angledelta':5,
                 'backtrack':True,
                 'gcr':gcr,
                 'cumulativesky':False,
                 'azimuth': sazm,
                 'fixed_tilt_angle': fixed_tilt_angle,
                 }



In [15]:
trackerdict = radObj.set1axis(**trackerParams)


In [16]:
# -- generate sky   
trackerdict = radObj.gendaylit1axis()
print(trackerdict)
print("LEN TRACKERDICT", len(trackerdict.keys()))


Creating ~2 skyfiles. 
Created 2 skyfiles in /skies/
{'2021-11-08_0930': {'surf_azm': 90.0, 'surf_tilt': 47.37, 'theta': -47.37, 'dni': 778.0, 'ghi': 271.0, 'dhi': 55.0, 'temp_air': -1.0, 'wind_speed': 2.5, 'skyfile': 'skies\\sky2_44.25_-117.34_2021-11-08_0930.rad'}, '2021-11-08_1030': {'surf_azm': 90.0, 'surf_tilt': 50.0, 'theta': -50.0, 'dni': 876.0, 'ghi': 405.0, 'dhi': 65.0, 'temp_air': 1.0, 'wind_speed': 2.5, 'skyfile': 'skies\\sky2_44.25_-117.34_2021-11-08_1030.rad'}}
LEN TRACKERDICT 2


In [17]:
sceneDict = {'pitch':pitch, 
             'hub_height': hub_height,
             'nMods': 19,
             'nRows': 7,
            'tilt': fixed_tilt_angle,  
            'sazm': sazm
             }



In [18]:
trackerdict = radObj.makeScene1axis(module=modulename,sceneDict=sceneDict)



Making ~2 .rad files for gendaylit 1-axis workflow (this takes a minute..)
2 Radfiles created in /objects/


In [19]:
trackerdict = radObj.makeOct1axis()



Making 2 octfiles in root directory.
Created 1axis_2021-11-08_0930.oct
Created 1axis_2021-11-08_1030.oct


In [21]:
trackerdict = radObj.analysis1axis(trackerdict, customname = 'Module',
                                   sensorsy=9)

Linescan in process: 1axis_2021-11-08_0930Module_Scene0_Row4_Module10_Front
Linescan in process: 1axis_2021-11-08_0930Module_Scene0_Row4_Module10_Back
Saved: results\irr_1axis_2021-11-08_0930Module_Scene0_Row4_Module10.csv
Index: 2021-11-08_0930. Wm2Front: 601.1345666666667. Wm2Back: 13.30378
Linescan in process: 1axis_2021-11-08_1030Module_Scene0_Row4_Module10_Front
Linescan in process: 1axis_2021-11-08_1030Module_Scene0_Row4_Module10_Back
Saved: results\irr_1axis_2021-11-08_1030Module_Scene0_Row4_Module10.csv
Index: 2021-11-08_1030. Wm2Front: 617.2475444444444. Wm2Back: 35.205977777777775


In [24]:
compiledresults = radObj.calculateResults(bifacialityfactor=0.7, agriPV=False)


No CECModule data passed; using default for Prism Solar BHC72-400


c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:64: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  alpha_sc=float(CECMod.alpha_sc),
c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:65: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_ref=float(CECMod.a_ref),
c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  I_L_ref=float(CECMod.I_L_ref),
c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:67: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(

In [32]:
compiledresults.keys()

Index(['timestamp', 'name', 'modNum', 'rowNum', 'sceneNum', 'x', 'y', 'z',
       'mattype', 'rearMat', 'Wm2Front', 'Wm2Back', 'backRatio', 'module',
       'row', 'sceneNum', 'POA_eff', 'Grear_mean', 'Gfront_mean', 'Pout_raw',
       'Pout_Gfront', 'BGG', 'BGE', 'Mismatch', 'Pout', 'Wind Speed', 'DNI',
       'DHI', 'GHI'],
      dtype='object')

In [33]:
ResultDatetime = list(radObj.CompiledResults['timestamp']) 
ResultPVWm2Back = list(radObj.CompiledResults['Grear_mean'])
ResultPVWm2Front = list(radObj.CompiledResults['Gfront_mean'])
ResultGHI = list(radObj.CompiledResults['GHI'])
ResultDNI = list(radObj.CompiledResults['DNI'])
ResultDHI = list(radObj.CompiledResults['DHI'])
ResultPout = list(radObj.CompiledResults['Pout'])
ResultWindSpeed = list(radObj.CompiledResults['Wind Speed'])


In [31]:
# NOT WORKING
ResultModuleTemp = list(radObj.CompiledResults['Module_temp'])
ResultModuleTemp

KeyError: 'Module_temp'

In [36]:
# Modify modscanfront for Ground
resolutionGround = 0.1  # use 1 for faster test runs
numsensors = int((pitch/resolutionGround)+1)
modscanfront = {'xstart': 0, 
                'zstart': 0.05,
                'xinc': resolutionGround,
                'zinc': 0,
                'Ny':numsensors,
                'orient':'0 0 -1'}

# Analysis for GROUND
trackerdict = radObj.analysis1axis(customname = 'Ground',
                                   modscanfront=modscanfront, sensorsy=1)



Linescan in process: 1axis_2021-11-08_0930Ground_Scene0_Row4_Module10_Front
Linescan in process: 1axis_2021-11-08_0930Ground_Scene0_Row4_Module10_Back
Saved: results\irr_1axis_2021-11-08_0930Ground_Scene0_Row4_Module10_Front.csv
Saved: results\irr_1axis_2021-11-08_0930Ground_Scene0_Row4_Module10_Back.csv
Index: 2021-11-08_0930. Wm2Front: 26.987325098039214. Wm2Back: 13.040820000000002
Linescan in process: 1axis_2021-11-08_1030Ground_Scene0_Row4_Module10_Front
Linescan in process: 1axis_2021-11-08_1030Ground_Scene0_Row4_Module10_Back
Saved: results\irr_1axis_2021-11-08_1030Ground_Scene0_Row4_Module10_Front.csv
Saved: results\irr_1axis_2021-11-08_1030Ground_Scene0_Row4_Module10_Back.csv
Index: 2021-11-08_1030. Wm2Front: 154.44611588235293. Wm2Back: 32.49606


In [38]:
compiled2 = radObj.calculateResults(bifacialityfactor=0.7, agriPV=True)


No CECModule data passed; using default for Prism Solar BHC72-400


c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:64: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  alpha_sc=float(CECMod.alpha_sc),
c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:65: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_ref=float(CECMod.a_ref),
c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  I_L_ref=float(CECMod.I_L_ref),
c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:67: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [41]:
trackerdict

{'2021-11-08_0930': {'surf_azm': 90.0,
  'surf_tilt': 47.37,
  'theta': -47.37,
  'dni': 778.0,
  'ghi': 271.0,
  'dhi': 55.0,
  'temp_air': -1.0,
  'wind_speed': 2.5,
  'skyfile': 'skies\\sky2_44.25_-117.34_2021-11-08_0930.rad',
  'scenes': [{'module': {'x': 1, 'y': 2, 'z': 0.02, 'modulematerial': 'black', 'scenex': 1.01, 'sceney': 2.0, 'scenez': 0.1, 'numpanels': 1, 'bifi': 1, 'text': '! genbox black PVmodule 1 2 0.02 | xform -t -0.5 -1.0 0 -a 1 -t 0 2.0 0', 'modulefile': 'objects\\PVmodule.rad', 'glass': False, 'offsetfromaxis': 0, 'xgap': 0.01, 'ygap': 0.0, 'zgap': 0.1}, 'modulefile': 'objects\\PVmodule.rad', 'hpc': True, 'name': 'Scene0', 'gcr': 0.4, 'text': '!xform -rx 47.37 -t 0 0 1.5 -a 19 -t 1.01 0 0 -a 7 -t 0 5 0 -i 1 -t -9.09 -15.0 0 -rz 90.0 -t 0 0 0 "C:\\Users\\sayala\\Documents\\GitHub\\InSPIRE\\Studies\\USMap_Doubleday_2024\\TEMP\\objects\\PVmodule.rad"', 'radfiles': 'C:\\Users\\sayala\\Documents\\GitHub\\InSPIRE\\Studies\\USMap_Doubleday_2024\\TEMP\\objects\\1axis2021-1

In [67]:
list(trackerdict[key]['AnalysisObj'][1].Wm2Front)

[23.62369,
 24.75632,
 25.783329999999996,
 27.12949,
 28.19743,
 29.412320000000005,
 30.627220000000005,
 32.0546,
 32.99094,
 33.92729,
 34.654,
 35.02919,
 35.40437,
 36.71206,
 36.43124,
 36.15041,
 35.86958,
 35.58876,
 33.07798,
 32.60631,
 32.13464,
 31.66297,
 31.1913,
 30.71963,
 29.14254,
 28.652,
 28.16146,
 27.17035,
 26.67018,
 26.17001,
 25.07406,
 24.5251,
 23.97614,
 22.627709999999997,
 22.15004,
 21.67238,
 21.04867,
 20.49321,
 19.93776,
 19.44088,
 18.78777,
 18.13466,
 18.64796,
 18.40119,
 18.15442,
 20.26754,
 20.66636,
 21.35884,
 22.24459,
 22.99386,
 24.04883]

In [75]:
keys=list(trackerdict.keys())

ResultGroundIrrad = []
ResultTemp = []
for key in keys:
    # before version 0.4.2 comit 236 or something
    #ResultGroundIrrad.append(trackerdict[key]['Results'][1]['Wm2Front'])
    ResultGroundIrrad.append(list(trackerdict[key]['AnalysisObj'][1].Wm2Front))
    ResultTemp.append(trackerdict[key]['temp_air']) # TEMP AIR NOR SAVED ON COMPILED RESULTS OR ELSEWHERE


In [20]:
sceneDict = {'pitch':pitch, 
             'hub_height': hub_height,
             'nMods': 19,
             'nRows': 7,
            'tilt': fixed_tilt_angle,  
            'sazm': sazm
             }

trackerdict = radObj.makeScene1axis(module=modulename,sceneDict=sceneDict)
